# Project 03: Iris Species Classification

**Difficulty**: ⭐ Beginner

**Estimated Time**: 15-20 hours

**Project Type**: Multi-class Classification

**Dataset**: Iris Dataset (Classic ML Dataset)

## Learning Objectives

By the end of this project, you will be able to:
1. Perform multi-class classification (vs binary classification)
2. Visualize high-dimensional data with PCA and pair plots
3. Compare multiple classification algorithms systematically
4. Understand model decision boundaries
5. Apply dimensionality reduction techniques

## Problem Statement

The Iris dataset is a classic dataset in machine learning, introduced by Ronald Fisher in 1936. It contains measurements of iris flowers from three different species.

**Goal**: Classify iris flowers into three species (setosa, versicolor, virginica) based on sepal and petal measurements.

## Prerequisites

- Machine Learning Fundamentals
- Classification algorithms
- Data Visualization
- Basic statistics

## 1. Setup and Imports

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# ML Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

# Configure visualization
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ All libraries imported successfully!")

## 2. Load and Inspect Data

In [ ]:
# Load Iris dataset
iris = load_iris(as_frame=True)

# Create DataFrame
df = iris.frame

print(f"Dataset shape: {df.shape}")
print(f"Features: {iris.feature_names}")
print(f"Target: {iris.target_names}")
print("\n" + "="*50)

# Display first few rows
df.head()

In [ ]:
# Dataset information
df.info()

In [ ]:
# Statistical summary
df.describe()

In [ ]:
# Class distribution
print("Class Distribution:")
class_counts = df['target'].value_counts().sort_index()
for idx, count in class_counts.items():
    print(f"{iris.target_names[idx]}: {count} samples")

# Visualize
plt.figure(figsize=(10, 5))
plt.bar([iris.target_names[i] for i in class_counts.index], class_counts.values,
        color=['#FF6B6B', '#4ECDC4', '#95E1D3'])
plt.xlabel('Species')
plt.ylabel('Count')
plt.title('Class Distribution (Perfectly Balanced)')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ Dataset is perfectly balanced!")

## 3. Exploratory Data Analysis

### 3.1 Pair Plot

In [ ]:
# Create pair plot to visualize all feature relationships
sns.pairplot(df, hue='target', palette=['#FF6B6B', '#4ECDC4', '#95E1D3'],
             diag_kind='hist', markers=['o', 's', 'D'])
plt.suptitle('Iris Dataset - Pair Plot by Species', y=1.02)
plt.tight_layout()
plt.show()

print("📊 Key Observations:")
print("- Setosa (class 0) is clearly separable")
print("- Versicolor and Virginica have some overlap")
print("- Petal features show better separation than sepal features")

### 3.2 Box Plots by Feature

In [ ]:
# Box plots for each feature by species
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

feature_names = iris.feature_names
colors = ['#FF6B6B', '#4ECDC4', '#95E1D3']

for idx, feature in enumerate(feature_names):
    data_to_plot = [df[df['target'] == i][feature] for i in range(3)]
    bp = axes[idx].boxplot(data_to_plot, labels=iris.target_names,
                           patch_artist=True)
    
    # Color boxes
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
    
    axes[idx].set_title(feature)
    axes[idx].set_ylabel('Measurement (cm)')
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 3.3 Correlation Heatmap

In [ ]:
# Correlation matrix
correlation_matrix = df.iloc[:, :-1].corr()  # Exclude target

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

print("📊 Key Observations:")
print("- High correlation between petal length and petal width (0.96)")
print("- High correlation between sepal length and petal length/width")
print("- Sepal width has low correlation with other features")

### 3.4 Principal Component Analysis (PCA)

In [ ]:
# Prepare data for PCA
X = df.iloc[:, :-1]
y = df['target']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Explained variance
explained_var = pca.explained_variance_ratio_
print(f"Explained variance by PC1: {explained_var[0]:.2%}")
print(f"Explained variance by PC2: {explained_var[1]:.2%}")
print(f"Total explained variance: {sum(explained_var):.2%}")

# Visualize PCA
plt.figure(figsize=(10, 6))
colors_map = {0: '#FF6B6B', 1: '#4ECDC4', 2: '#95E1D3'}
for i, species in enumerate(iris.target_names):
    mask = y == i
    plt.scatter(X_pca[mask, 0], X_pca[mask, 1],
               label=species, color=colors_map[i], alpha=0.7, s=100)

plt.xlabel(f'First Principal Component ({explained_var[0]:.1%} variance)')
plt.ylabel(f'Second Principal Component ({explained_var[1]:.1%} variance)')
plt.title('Iris Dataset - PCA Projection')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 PCA captures 95.8% of variance with just 2 components!")

## 4. Data Preparation

In [ ]:
# Prepare features and target
X = df.iloc[:, :-1].values
y = df['target'].values

# Split data (70/30 split - smaller dataset)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nClass distribution in training set:")
unique, counts = np.unique(y_train, return_counts=True)
for cls, count in zip(unique, counts):
    print(f"  {iris.target_names[cls]}: {count}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✅ Data prepared and scaled successfully!")

## 5. Model Training and Evaluation

In [ ]:
# Define models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'SVM': SVC(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB()
}

# Train and evaluate
results = {}

print("Training models...\n")
for name, model in models.items():
    # Use scaled data for models that benefit from it
    if name in ['Logistic Regression', 'SVM', 'K-Nearest Neighbors']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Calculate metrics (weighted for multi-class)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    results[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'model': model,
        'predictions': y_pred
    }
    
    print(f"✅ {name}: Accuracy={accuracy:.4f}, F1={f1:.4f}")

print("\n✅ All models trained successfully!")

In [ ]:
# Compare model performance
results_df = pd.DataFrame({
    name: {metric: values[metric] for metric in ['Accuracy', 'Precision', 'Recall', 'F1 Score']}
    for name, values in results.items()
}).T

print("Model Performance Comparison:")
print(results_df.sort_values('Accuracy', ascending=False))

# Best model
best_model_name = results_df['Accuracy'].idxmax()
print(f"\n🏆 Best Model: {best_model_name} with {results_df.loc[best_model_name, 'Accuracy']:.2%} accuracy")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
colors = ['#4ECDC4', '#FF6B6B', '#95E1D3', '#FFD93D']

for idx, (metric, color) in enumerate(zip(metrics, colors)):
    sorted_results = results_df.sort_values(metric, ascending=True)
    sorted_results[metric].plot(kind='barh', ax=axes[idx], color=color)
    axes[idx].set_xlabel(metric)
    axes[idx].set_title(f'Model Comparison: {metric}')
    axes[idx].set_xlim(0, 1)
    
    # Add value labels
    for i, v in enumerate(sorted_results[metric]):
        axes[idx].text(v + 0.01, i, f'{v:.3f}', va='center')

plt.tight_layout()
plt.show()

## 6. Detailed Analysis - Best Model

In [ ]:
# Confusion matrix for best model
best_predictions = results[best_model_name]['predictions']
cm = confusion_matrix(y_test, best_predictions)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True,
            xticklabels=iris.target_names,
            yticklabels=iris.target_names)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title(f'Confusion Matrix - {best_model_name}')
plt.tight_layout()
plt.show()

# Classification report
print(f"\nDetailed Classification Report - {best_model_name}:")
print(classification_report(y_test, best_predictions, target_names=iris.target_names))

## 7. Cross-Validation

In [ ]:
# 5-fold cross-validation on all models
print("Performing 5-fold cross-validation...\n")

cv_results = {}

for name, model_info in results.items():
    model = model_info['model']
    
    # Use scaled data for appropriate models
    if name in ['Logistic Regression', 'SVM', 'K-Nearest Neighbors']:
        scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    else:
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
    cv_results[name] = {
        'Mean': scores.mean(),
        'Std': scores.std()
    }
    
    print(f"{name}: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

# Visualize
cv_df = pd.DataFrame(cv_results).T.sort_values('Mean', ascending=True)

plt.figure(figsize=(10, 6))
plt.barh(cv_df.index, cv_df['Mean'], xerr=cv_df['Std'], color='#4ECDC4', alpha=0.7)
plt.xlabel('Mean Cross-Validation Accuracy')
plt.title('5-Fold Cross-Validation Results')
plt.xlim(0, 1)

for i, (idx, row) in enumerate(cv_df.iterrows()):
    plt.text(row['Mean'] + 0.01, i, f"{row['Mean']:.3f}", va='center')

plt.tight_layout()
plt.show()

## 8. Key Insights and Conclusions

### Summary of Findings

**1. Dataset Characteristics:**
- 150 samples, 4 features, 3 classes (perfectly balanced)
- Setosa is linearly separable from other species
- Versicolor and Virginica have some overlap
- Petal features are more discriminative than sepal features

**2. Feature Importance:**
- Petal length and petal width are highly correlated (r=0.96)
- Petal features show better class separation
- PCA with 2 components captures 95.8% of variance

**3. Model Performance:**
- Most models achieved >95% accuracy (simple problem)
- Logistic Regression, SVM, and Random Forest: near-perfect performance
- Decision Tree and KNN: slight overfitting on small dataset
- Cross-validation shows consistent performance

**4. Multi-Class Classification:**
- Confusion matrices show most errors between Versicolor and Virginica
- Setosa is perfectly classified by all models
- Weighted F1 scores account for class balance

**5. Insights:**
- This is a "toy" dataset - too simple for real-world complexity
- Demonstrates multi-class classification principles
- Good for understanding model behavior and visualization
- Linear models work well when data is linearly separable

## 9. Project Checklist

✅ **Completed:**
- [x] Data loading and inspection
- [x] Class distribution analysis
- [x] Pair plot visualization
- [x] Box plots by feature
- [x] Correlation analysis
- [x] PCA dimensionality reduction
- [x] Train-test split
- [x] Feature scaling
- [x] 6 classification models
- [x] Model comparison
- [x] Confusion matrix
- [x] Classification report
- [x] Cross-validation
- [x] Insights and conclusions

📋 **For Portfolio:**
- [ ] Create README.md
- [ ] Add requirements.txt
- [ ] Create decision boundary visualizations
- [ ] Compare with other dimensionality reduction (t-SNE, UMAP)
- [ ] Add to portfolio website